<h1>导入数据</h1>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import time
import tensorflow.keras.backend as K
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from tensorflow.keras.layers import Input, Dense, LSTM, MaxPool1D, Dropout, concatenate, Flatten, Conv1D
from tensorflow.keras.callbacks import History, TensorBoard, EarlyStopping
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from time import strftime, localtime

<h1>数据处理</h1>

<h2>数据预处理</h2>

<h3>conf_info数据预处理</h3>

In [2]:
cons_info = pd.read_csv("../data/训练组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info

,ID,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE,IS_FLAG
,,,,,,,,,,,,,,
1,329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
2,329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
3,329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
4,329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
5,329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8836,2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,NaN,120.0,2019/12/2 16:12:00,非临时用电,NaN,0
8837,2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,NaN,120.0,2019/12/3 16:33:00,非临时用电,NaN,0
8838,2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,NaN,120.0,2019/12/18 14:42:00,非临时用电,NaN,0


In [3]:
cons_info = cons_info.set_index("ID").drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,120.0,2019/12/2 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,120.0,2019/12/3 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,120.0,2019/12/18 14:42:00,非临时用电,0


In [4]:
cons_info["BUILD_DATE"] = pd.to_datetime(cons_info["BUILD_DATE"])
cons_info["LAST_CHK_DATE"] = pd.to_datetime(cons_info["LAST_CHK_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [5]:
cons_info["SHIFT_NO"] = cons_info["SHIFT_NO"].fillna(0)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [6]:
cons_info["now"] = "2021-12-31"
cons_info["now"] = pd.to_datetime(cons_info["now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now
ID,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31


In [7]:
cons_info["live_days"] = (cons_info["now"] - cons_info["BUILD_DATE"]).map(lambda x: x.days)
cons_info["check_days"] = (cons_info["now"] - cons_info["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now,live_days,check_days
ID,,,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31,743,743


In [8]:
cons_info = cons_info.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,IS_FLAG,live_days,check_days
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,240.0,非临时用电,0,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,120.0,非临时用电,0,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,743,743


In [9]:
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  8840 non-null   object 
 1   VOLT_NAME       8840 non-null   object 
 2   PRC_NAME        8840 non-null   object 
 3   CONTRACT_CAP    8840 non-null   float64
 4   RUN_CAP         8840 non-null   float64
 5   SHIFT_NO        8840 non-null   float64
 6   CHK_CYCLE       8838 non-null   float64
 7   TMP_NAME        8840 non-null   object 
 8   IS_FLAG         8840 non-null   int64  
 9   live_days       8840 non-null   int64  
 10  check_days      8840 non-null   int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 828.8+ KB


In [10]:
cons_info = pd.get_dummies(cons_info)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,0,0,0,0,0,0,0,1,0,1,1
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,0,0,0,0,0,0,1,0,0,1,1
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,0,0,0,0,0,0,0,1,0,1,1


<h3>elec_month数据预处理</h3>

In [11]:
elec_month = pd.read_csv("../data/训练组_比特币挖矿_月用电明细（20211217）.csv")
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [12]:
elec_month["ym"] = elec_month["ym"].astype(str)
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [13]:
elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]] = elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]] / elec_month[["pq_f", "pq_g", "pq_p", "pq_z"]].max(axis=0)
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,0.018825,0.009651,0.000000,0.010450
1,329769614,202002,0.015060,0.009232,-0.000036,0.008686
2,329769614,202003,0.013988,0.006834,0.000036,0.007703
3,329769614,202004,0.015978,0.010251,0.000000,0.009332
4,329769614,202005,0.015886,0.010850,0.000000,0.009426
...,...,...,...,...,...,...
194475,2878334859,202106,0.003275,0.004016,-0.000036,0.002330
194476,2878334859,202107,0.005204,0.006714,0.000036,0.003811
194477,2878334859,202108,0.003122,0.004016,0.000000,0.002276
194478,2878334859,202109,0.003428,0.003896,-0.000036,0.002370


In [14]:
cons_month_result = []
for cons_id in tqdm(cons_info.index):
    df = elec_month[elec_month["id"] == cons_id]
    df1 = df.drop(columns=["id"]).sort_values("ym").drop(columns=["ym"])
    cons_month_result.append(df1.values)
cons_month = np.array(cons_month_result)
cons_month.shape

100%|█████████████████████████████████| 8840/8840 [00:13<00:00, 677.86it/s]


(8840, 22, 4)

<h3>elec_day数据预处理</h3>

In [15]:
elec_day = pd.read_csv("../data/训练组_比特币挖矿_日用电明细（20211217）.csv")
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23 00:00:00,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24 00:00:00,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25 00:00:00,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26 00:00:00,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06 00:00:00,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07 00:00:00,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08 00:00:00,7.23,NaN,0.0,4.50,0.0,2.73


In [16]:
elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] = elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]] / elec_day[["kwh", "kwh_rap", "kwh_pap_r1", "kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]].max(axis=0)
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,0.000367,0.0,0.0,0.000641,0.0,0.000450
1,329449398,2020-01-23 00:00:00,0.000516,0.0,0.0,0.000752,0.0,0.000811
2,329449398,2020-01-24 00:00:00,0.000339,0.0,0.0,0.000478,0.0,0.000553
3,329449398,2020-01-25 00:00:00,0.000340,0.0,0.0,0.000562,0.0,0.000457
4,329449398,2020-01-26 00:00:00,0.000338,0.0,0.0,0.000548,0.0,0.000466
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,0.000225,NaN,0.0,0.000389,0.0,0.000282
954565,2878334859,2021-10-06 00:00:00,0.000225,NaN,0.0,0.000388,0.0,0.000283
954566,2878334859,2021-10-07 00:00:00,0.000225,NaN,0.0,0.000388,0.0,0.000282
954567,2878334859,2021-10-08 00:00:00,0.000225,NaN,0.0,0.000390,0.0,0.000282


In [17]:
elec_day = elec_day.fillna(0)
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,0.000367,0.0,0.0,0.000641,0.0,0.000450
1,329449398,2020-01-23 00:00:00,0.000516,0.0,0.0,0.000752,0.0,0.000811
2,329449398,2020-01-24 00:00:00,0.000339,0.0,0.0,0.000478,0.0,0.000553
3,329449398,2020-01-25 00:00:00,0.000340,0.0,0.0,0.000562,0.0,0.000457
4,329449398,2020-01-26 00:00:00,0.000338,0.0,0.0,0.000548,0.0,0.000466
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,0.000225,0.0,0.0,0.000389,0.0,0.000282
954565,2878334859,2021-10-06 00:00:00,0.000225,0.0,0.0,0.000388,0.0,0.000283
954566,2878334859,2021-10-07 00:00:00,0.000225,0.0,0.0,0.000388,0.0,0.000282
954567,2878334859,2021-10-08 00:00:00,0.000225,0.0,0.0,0.000390,0.0,0.000282


In [18]:
dates = pd.to_datetime(elec_day["rq"].unique()).sort_values()

In [19]:
cons_day_result = []
for cons_id in tqdm(cons_info.index):
    df = elec_day[elec_day["id"] == cons_id].drop(columns=["id"]).sort_values("rq")
    df["rq"] = pd.to_datetime(df["rq"])
    df = df.set_index("rq")
    df = df.reindex(dates).fillna(0)
    cons_day_result.append(df.values)
cons_day = np.array(cons_day_result)
cons_day.shape

100%|█████████████████████████████████| 8840/8840 [00:23<00:00, 369.95it/s]


(8840, 108, 6)

<h2>查看处理好的结果</h2>

<h3>标签</h3>

In [20]:
target = cons_info["IS_FLAG"].values.astype(np.float64)

<h3>统一视图</h3>

In [21]:
cons_info = cons_info.drop(columns=["IS_FLAG"]).values
cons_info.shape

(8840, 20)

<h3>月均电量</h3>

In [22]:
cons_month.shape

(8840, 22, 4)

<h3>日电量</h3>

In [23]:
cons_day.shape

(8840, 108, 6)

<h2>对正样本重复</h2>

In [24]:
cons_info_pos = cons_info[target == 1].repeat(800, axis=0)
cons_info_pos.shape

(8000, 20)

In [25]:
cons_month_pos = cons_month[target == 1].repeat(800, axis=0)
cons_month_pos.shape

(8000, 22, 4)

In [26]:
cons_day_pos = cons_day[target == 1].repeat(800, axis=0)
cons_day_pos.shape

(8000, 108, 6)

<h2>提取负样本</h2>

In [27]:
cons_info_neg = cons_info[target == 0]
cons_month_neg = cons_month[target == 0]
cons_day_neg = cons_day[target == 0]
print(cons_info_neg.shape)
print(cons_month_neg.shape)
print(cons_day_neg.shape)

(8830, 20)
(8830, 22, 4)
(8830, 108, 6)


<h2>合并正负样本</h2>

In [28]:
cons_info = np.concatenate([cons_info_pos, cons_info_neg], axis=0)
cons_month = np.concatenate([cons_month_pos, cons_month_neg], axis=0)
cons_day = np.concatenate([cons_day_pos, cons_day_neg], axis=0)
print(cons_info.shape)
print(cons_month.shape)
print(cons_day.shape)

(16830, 20)
(16830, 22, 4)
(16830, 108, 6)


<h2>重复标签</h2>

In [29]:
target_pos = target[target == 1]
target_neg = target[target == 0]
target_pos = target_pos.repeat(800, axis=0)
target = np.concatenate([target_pos, target_neg], axis=0)
print(target.shape)

(16830,)


<h1>构建模型</h1>

<h2>初始化tensorflow</h2>

In [30]:
clear_session()
tf.compat.v1.reset_default_graph()
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

<h2>构建模型</h2>

In [31]:
def build_model(cons_info_shape, cons_month_shape, cons_day_shape) -> Model:
    input_cons_info = Input(shape=cons_info_shape, name="input_cons_info")
    input_cons_month = Input(shape=cons_month_shape, name="input_cons_month")
    input_cons_day = Input(shape=cons_day_shape, name="input_cons_day")
    
    x1 = Dense(units=32, activation="relu", name="x1")(input_cons_info)
    x2 = Dense(units=64, activation="relu", name="x2")(x1)
    x3 = Dropout(0.2, name="x3")(x2)
    
    y11 = Conv1D(filters=16, kernel_size=5, strides=1, padding="causal", activation="relu", name="conv1d_y1")(input_cons_month)
    y12 = MaxPool1D(pool_size=5, strides=1, padding="valid", name="max_pool1d_y1")(y11)
    y13 = Conv1D(filters=32, kernel_size=3, strides=1, padding="causal", activation="relu", name="conv1d_y2")(y12)
    y14 = MaxPool1D(pool_size=3, strides=1, padding="valid", name="max_pool1d_y2")(y13)
    y1, y_hidden_state, y_cell_state = LSTM(units=16, return_sequences=True, return_state=True, name="y1")(y14)
    y_hidden_state_1 = Dense(units=64, name="y_hidden_state_1")(y_hidden_state)
    y_cell_state_1 = Dense(units=64, name="y_cell_state_1")(y_cell_state)
    y2 = LSTM(units=64, return_sequences=True, return_state=False, name="y2")(y1, [y_hidden_state_1, y_cell_state_1])
    
    z11 = Conv1D(filters=16, kernel_size=5, strides=1, padding="causal", activation="relu", name="conv1d_z1")(input_cons_day)
    z12 = MaxPool1D(pool_size=5, strides=1, padding="valid", name="max_pool1d_z1")(z11)
    z13 = Conv1D(filters=32, kernel_size=3, strides=1, padding="causal", activation="relu", name="conv1d_z2")(z12)
    z14 = MaxPool1D(pool_size=3, strides=1, padding="valid", name="max_pool1d_z2")(z13)
    z1, z_hidden_state, z_cell_state = LSTM(units=16, return_sequences=True, return_state=True, name="z1")(z14)
    z_hidden_state_1 = Dense(units=64, name="z_hidden_state_1")(z_hidden_state)
    z_cell_state_1 = Dense(units=64, name="z_cell_state_1")(z_cell_state)
    z2 = LSTM(units=64, return_sequences=True, return_state=False, name="z2")(z1, [z_hidden_state_1, z_cell_state_1])
    
    s1 = concatenate([y2, z2], axis=1, name="s1")
    s2 = LSTM(units=16, return_sequences=True, return_state=False, name="s2")(s1)
    s3 = LSTM(units=8, return_sequences=True, return_state=False, name="s3")(s2)
    s4 = Flatten(name="s4")(s3)
    s5 = Dropout(0.2, name="s5")(s4)
    
    o1 = concatenate([x3, s5], axis=-1, name="o1")
    o2 = Dense(units=64, name="o2")(o1)
    o3 = Dense(units=16, name="o3")(o2)
    o4 = Dropout(0.2, name="o4")(o3)
    o5 = Dense(units=1, name="o5")(o4)
    
    result = Model(inputs=[input_cons_info, input_cons_month, input_cons_day], outputs=[o4], name="mining-detector")
    result.summary()
    return result

In [32]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [33]:
model = build_model(cons_info.shape[1:], cons_month.shape[1:], cons_day.shape[1:])
model.compile(
    optimizer='adam', 
    loss="binary_crossentropy", 
    metrics=["accuracy", recall_m, precision_m, f1_m]
)

Model: "mining-detector"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_cons_month (InputLayer)   [(None, 22, 4)]      0                                            
__________________________________________________________________________________________________
input_cons_day (InputLayer)     [(None, 108, 6)]     0                                            
__________________________________________________________________________________________________
conv1d_y1 (Conv1D)              (None, 22, 16)       336         input_cons_month[0][0]           
__________________________________________________________________________________________________
conv1d_z1 (Conv1D)              (None, 108, 16)      496         input_cons_day[0][0]             
____________________________________________________________________________________

<h2>训练模型</h2>

<h3>设置class_weight和sample_weight</h3>

In [34]:
class_weight = compute_class_weight('balanced', np.unique(target) ,target)
class_weights = dict()
for class_name, weight in enumerate(class_weight):
    class_weights[class_name] = weight
    
sample_weights = np.ones(shape=(len(target),))
sample_weights[target == 1] = 800

c:\users\drzon\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[1. 1. 1. ... 0. 0. 0.] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


<h3>开始训练</h3>

In [35]:
model.fit(
    [cons_info, cons_month, cons_day], 
    target, 
    validation_split=0.2,
    epochs=100, 
    batch_size=32, 
    use_multiprocessing=True, 
    workers=12,
    callbacks=[
        TensorBoard(log_dir="../logs/%s" % strftime("%Y%m%d_%H.%M.%S", localtime()), write_graph=True, write_images=True),
        EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, restore_best_weights=True)
    ]
)

Epoch 1/100
  1/421 [..............................] - ETA: 0s - loss: 7.4433 - accuracy: 0.0000e+00 - recall_m: 5.6250 - precision_m: 0.5233 - f1_m: 0.9574WARNING:tensorflow:From c:\users\drzon\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
421/421 [==============================] - 13s 31ms/step - loss: 7.5728 - accuracy: 0.0000e+00 - recall_m: 8.5569 - precision_m: 0.5941 - f1_m: 1.1094 - val_loss: 10.4838 - val_accuracy: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/100
421/421 [==============================] - 11s 26ms/step - loss: 7.4368 - accuracy: 0.0000e+00 - recall_m: 9.2811 - precision_m: 0.5943 - f1_m: 1.1155 - val_loss: 11.4358 - val_accuracy: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: